# FINM3422 Coding Project 2: OTC Derivatives Pricing & Hedging

This notebook implements an object-oriented solution to price four OTC derivative trades and calculate portfolio-level hedging parameters. All data and valuations are as of close of market on **16 May 2025**. (update this text, add anything else? Introduce task)

## Imports and Setup

In [ ]:
from Option_Classes import EuropeanOption
from Option_Classes import AmericanPutOption
from Option_Classes import UpAndInCallOption
from Option_Classes import BasketCallOption
from Discount_Rate import get_discount_factor
from Discount_Rate import get_zero_rate
from Hedging_Parameters import HedgingCalculator
from market_data import get_close_price
from market_data import get_volatility
from market_data import get_correlation_matrix
from market_data import get_all_market_data
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf
import importlib

## Market Data Inputs

In [ ]:
tickers = ["BHP.AX", "CBA.AX", "WES.AX", "CSL.AX", "WDS.AX", "MQG.AX"]
market_data = get_all_market_data(tickers, date="2025-05-16", window=60)

spot = market_data["spot"]
vol = market_data["vol"]
corr = market_data["corr"]

## broken asf atm

## Discount Curve Construction

do we build discount curve in the .ipynb file, or can we just produce the discount rates in the .py file?

In [ ]:

# This module builds a discount curve from bank bills and bonds, and provides functions to retrieve discount factors and zero rates
from instrument_classes import Bank_bill, Bond, Portfolio
from curve_classes_and_functions import YieldCurve

# Builds bootstrapped discount curve from bank bills and bonds. 

def build_discount_curve():
    # Step 1: Create instruments. Data pulled from Bloomberg. 
    bill1 = Bank_bill(face_value=100, maturity=0.25, price=98.5)
    bill2 = Bank_bill(face_value=100, maturity=0.5, price=97.3)
    bill1.set_cash_flows()
    bill2.set_cash_flows()

    bond1 = Bond(face_value=100, maturity=1.0, coupon=0.04, frequency=4, price=99.2)
    bond2 = Bond(face_value=100, maturity=2.0, coupon=0.045, frequency=4, price=98.7)
    bond3 = Bond(face_value=100, maturity=3.0, coupon=0.05, frequency=4, price=97.0)
    bond1.set_cash_flows()
    bond2.set_cash_flows()
    bond3.set_cash_flows()

    # Step 2: Add to portfolio
    portfolio = Portfolio()
    portfolio.add_bank_bill(bill1)
    portfolio.add_bank_bill(bill2)
    portfolio.add_bond(bond1)
    portfolio.add_bond(bond2)
    portfolio.add_bond(bond3)
    portfolio.set_cash_flows()

    # Step 3: Bootstrap yield curve
    yc = YieldCurve()
    yc.set_constituent_portfolio(portfolio)
    yc.bootstrap()

    return yc

# Retrieves the discount factor for a given maturity from the bootstrapped yield curve
def get_discount_factor(yield_curve, maturity):
    return yield_curve.get_discount_factor(maturity)

# Returns the interpolated zero rate for a given maturity from the yield curve
def get_zero_rate(yield_curve, maturity):
    return yield_curve.get_zero_rate(maturity)



## Trade 1: BHP European Call

description of the trade

### Market Data

In [141]:
# Extract relevant data for option pricing and hedging
importlib.reload(market_data)
from market_data import get_close_price, get_volatility, get_correlation_matrix, get_all_market_data



ticker = "BHP.AX"
date = "2025-05-16"

spot_bhp = get_close_price(ticker, date) #good
vol_bhp = get_volatility(ticker, end_date=date, window=60) #good
strike_bhp = 0.98 * spot_bhp

def time_to_expiry(start, end):
    return (end - start).days / 365.0

expiry_bhp = time_to_expiry(datetime(2025, 5, 16), datetime(2027, 9, 15)) # 2.33 Years



print(f"Spot Price: {spot_bhp}, Volatility: {vol_bhp}, Strike Price: {strike_bhp}, Expiry: {expiry_bhp} years")

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


39.720001220703125
Ticker
BHP.AX    0.227511
dtype: float64


[*********************100%***********************]  1 of 1 completed

Spot Price: 39.720001220703125, Volatility: Ticker
BHP.AX    0.296738
dtype: float64, Strike Price: 38.925601196289065, Expiry: 2.334246575342466 years


### Discount Rate

In [142]:
# Given the time to expiry, find the discount factor and zero rate
rate_bhp = get_zero_rate(expiry_bhp)

print(f"Zero Rate for BHP: {rate_bhp}")


TypeError: get_zero_rate() missing 1 required positional argument: 'maturity'

### Option Valuation

In [ ]:
# Instantiate and price the option
bhp_option = EuropeanOption(ticker=ticker,spot=spot_bhp, strike=strike_bhp, expiry=expiry_bhp, rate=rate_bhp, vol=vol_bhp, option_type="call")

print(f"Option Price: {bhp_option.price()}")


### Hedging Parameters

In [ ]:
# Use HedgingCalculator to get delta, gamma, vega, theta, rho

hc = HedgingCalculator()
greeks_bhp = hc.get_all_greeks(bhp_option, method="fd")

print("Greeks for BHP Option:")
for greek, value in greeks_bhp.items():
    print(f"{greek}: {value}")

### Hedging Strategy

## Trade 2: CBA American Put

description of the trade

### Market Data

In [ ]:
# spot = ..., vol = ..., strike = ..., expiry = ..., rate = ...

### Discount Rate

In [ ]:
# rate = yield_curve.get_zero_rate(...)

### Valuation Method

In [ ]:
# Instantiate and price the option

### Hedging Parameters

In [ ]:
# Use HedgingCalculator to get delta, gamma, vega, theta, rho

### Hedging Strategy

## Trade 3: WES Barrier Call

description of the trade

### Market Data

In [ ]:
# spot = ..., vol = ..., strike = ..., expiry = ..., rate = ...

### Discount Rate

In [ ]:
# rate = yield_curve.get_zero_rate(...)

### Valuation Method

In [ ]:
# Instantiate and price the option

### Hedging Parameters

In [ ]:
# Use HedgingCalculator to get delta, gamma, vega, theta, rho

### Hedging Strategy

## Trade 4: Basket Call

description of the trade

### Market Data

In [ ]:
# spot = ..., vol = ..., strike = ..., expiry = ..., rate = ...

### Discount Rate

In [ ]:
# rate = yield_curve.get_zero_rate(...)

### Valuation Method

In [ ]:
# Instantiate and price the option

### Hedging Parameters

In [ ]:
# Use HedgingCalculator to get delta, gamma, vega, theta, rho

### Hedging Strategy

## Portfolio Valuation and Hedging Summary

In [ ]:
# Collect all option prices and Greek exposures into summary DataFrame
# Sum portfolio value and Greek sensitivities

## Design and Model Justification

- Why Black-Scholes, Binomial Tree, and Monte Carlo were chosen
- Why finite difference for Greeks
- Why the bump sizes used
- Accuracy vs flexibility trade-offs

## Conclusion

- Modular, OOP-driven pricing system
- Accurate pricing and hedging of diverse OTC trades
- Integration of real market data and theory